# Ejercicio Naive Bayes

<br>1) Estimadores de media y desvío. Considere una variable aleatoria con distribucion gaussiana con media μ=0 y desvio σ=1.</br>

In [8]:
import pandas as pd
import numpy as np
import sklearn.naive_bayes as GaussianNB
import matplotlib.pyplot as plt
%matplotlib inline

<br>a) A partir de muestras de tamaño n (n=3, 5, 10, 20 y 100), estimar los parametros $\hat{\mu}$ y ${\hat{\sigma}}$ usando los estimadores de máxima verosimilitud y calcular $a={\hat{\mu}}-{\mu}$ y $b={\frac{\hat{\sigma}}{\sigma}}$. Repetir 1000 veces el experimento y calcular la media de los a y de los b para los diferentes valores de n propuestos.</br>

In [9]:
mu=0
sigma=1
df_1a=pd.DataFrame(columns=['n', 'a', 'b'])
for n in (3, 5, 10, 20, 100):
    A=list()
    B=list()
    for i in range(1000):
        x=np.random.normal(mu, sigma, n)
        est_mu = x.mean()
        est_sigma = x.var()  #este estimador esta sesgado
        A.append(est_mu-mu)
        B.append(est_sigma/sigma)
    df_1a=df_1a.append(pd.DataFrame([[n, np.mean(A), np.mean(B)]], columns=['n', 'a', 'b']), ignore_index=True)

df_1a.head()
        

,n,a,b
0,3,-0.018550,0.680864
1,5,-0.014748,0.795682
2,10,-0.006645,0.884497
3,20,-0.007797,0.944892
4,100,0.000461,0.996453


<br>b) Repetir el item anterior usando estimadores insesgados. ¿Qué observa?</br>

In [10]:
df_1b=pd.DataFrame(columns=['n', 'a', 'b (estimador insesgado)', 'b (estimador sesgado)'])
for n in (3, 5, 10, 20, 100):
    A=list()
    B_ss=list()
    B_cs=list()
    for i in range(1000):
        x=np.random.normal(mu, sigma, n)
        est_mu = x.mean()
        est_sigma_ss = x.var()*(n/(n-1)) #este estimador no esta sesgado 
        est_sigma_cs = x.var() 
        A.append(est_mu-mu)
        B_ss.append(est_sigma_ss/sigma)
        B_cs.append(est_sigma_cs/sigma)
    df_1b=df_1b.append(pd.DataFrame([[n, np.mean(A), np.mean(B_ss), np.mean(B_cs)]], columns=['n', 'a', 'b (estimador insesgado)', 'b (estimador sesgado)']), ignore_index=True)

df_1b.head()

,n,a,b (estimador insesgado),b (estimador sesgado)
0,3,0.009607,0.935255,0.623503
1,5,-0.005149,0.976304,0.781043
2,10,0.007568,0.993857,0.894471
3,20,-0.003960,1.008809,0.958369
4,100,-0.000361,1.001072,0.991061


El estimador insesgado es mas cercano a la varianza original ${\sigma}=1$ 

2)Usando los datos del archivo tennis.txt, entrenar a mano un clasificador discreto bayesiano ingenuo y usarlo para decidir si habrá partido de tennis un día con las siguientes características: sunny outlook, cool temperature, high himidity and strong wind

In [35]:
df_tennis = pd.read_csv('tennis.txt')
columns_df=['Day', 'Outlook', 'Temperature', 'Humidity', 'Wind', 'Play']
df_tennis

,Day,Temp.,Outlook,Humid.,Wind,Play
0,D1,Hot,Sunny,High,Weak,No
1,D2,Hot,Sunny,High,Strong,No
2,D3,Hot,Overcst,High,Weak,Yes
3,D4,Mild,Rain,High,Weak,Yes
4,D5,Cool,Rain,Normal,Weak,Yes
5,D6,Cool,Rain,Normal,Strong,No
6,D7,Cool,Overcst,Normal,Strong,Yes
7,D8,Mild,Sunny,High,Weak,No
8,D9,Cool,Sunny,Normal,Weak,Yes
9,D10,Mild,Rain,Normal,Weak,Yes


Primero hay que limpiar un poco los datos y ponerlos en un formato de tabla correcto.

In [ ]:
list_of_rows = list()
for event in range(14):
    row = list([1,2,3])
    for feature in range(int(len(columns_df))):
        row.append(df_tennis_aux.T[1+event+15*feature][0])
    list_of_rows.append()

df_tennis = pd.DataFrame(list_of_rows, columns = columns_df)

Calculo las probabilidades de que alguien juege basado en los datos de la tabla.

In [ ]:
p_yes = len(df_tennis[df_tennis['Play'] == 'Yes']['Play'])/len(df_tennis['Play'])
p_no = len(df_tennis[df_tennis['Play'] == 'No']['Play'])/len(df_tennis['Play'])  

Esta funcion crea un diccionario el cual separa las tuplas por clase (en este caso cada clase es "Yes" o "No"), este diccionario me va a ayudar mas adelante.

Primero, hay que calcular la probabilidad de cada "estado" del dia dependiendo si jugó o no, por ejemplo $p(Hot\hspace{0.1cm}|\hspace{0.1cm}Yes)={\frac{number\hspace{0.25cm}of\hspace{0.25cm}hot\hspace{0.25cm}days\hspace{0.25cm}and\hspace{0.25cm}Yes}{days\hspace{0.25cm}Yes}}$

In [115]:
prob_x_if_play = dict()
prob_x_if_not_play = dict()
for col in df_tennis.columns.drop(['Day', 'Play']):
    for state in df_tennis[col].unique():
        prob_x_if_play.update({state: len(df_tennis[(df_tennis[col]==state) & (df_tennis['Play']=='Yes')])/len(df_tennis[df_tennis['Play']=='Yes'])})
        prob_x_if_not_play.update({state: len(df_tennis[(df_tennis[col]==state) & (df_tennis['Play']=='No')])/len(df_tennis[df_tennis['Play']=='No'])})

In [118]:
print(prob_x_if_play,'\n' ,prob_x_if_not_play)

{'Hot': 0.2222222222222222, 'Mild': 0.4444444444444444, 'Cool': 0.3333333333333333, 'Sunny': 0.2222222222222222, 'Overcast': 0.4444444444444444, 'Rain': 0.3333333333333333, 'High': 0.3333333333333333, 'Normal': 0.6666666666666666, 'Weak': 0.6666666666666666, 'Strong': 0.3333333333333333} 
 {'Hot': 0.4, 'Mild': 0.4, 'Cool': 0.2, 'Sunny': 0.6, 'Overcast': 0.0, 'Rain': 0.4, 'High': 0.8, 'Normal': 0.2, 'Weak': 0.4, 'Strong': 0.6}


<br>Sea X una tupla con las diferentes condiciones de un cierto dia, por ejemplo ['Hot', 'Mild', 'Rain', 'Weak'] <br>
<br>La probabilidad si jugó o no un cierto dia dado el "estado" de este dia va a ser: $p(Yes\hspace{0.1cm}|\hspace{0.1cm}X)={\frac{p(X\hspace{0.1cm}|\hspace{0.1cm}Yes) * p(Yes)}{p(X)}}$ o $p(No\hspace{0.1cm}|\hspace{0.1cm}X)={\frac{p(X\hspace{0.1cm}|\hspace{0.1cm}No) * p(Yes)}{p(X)}}$ respectivamente.<br>
<br>Como la prediccion va a ser "Yes" o "No" dependiendo de cuales de las dos probabilidades es mayor, no hace falta dividir por el denominador P(X), ya que ambas probabilidades se obtienen dividiendo por lo mismo. <br>
<br>La siguiente funcion predice si alguien jugo (1) o no jugo (0) dependiendo del "estado" del dia que le pasemos como argumento<br>

In [125]:
def predict(state_of_day):
    prob_plays_if_SOF = 1
    prob_not_plays_if_SOF = 1
    for state in state_of_day:
        prob_plays_if_SOF *= prob_x_if_play[state] * p_yes
        prob_not_plays_if_SOF *= prob_x_if_not_play[state] * p_no
        
    if prob_plays_if_SOF > prob_not_plays_if_SOF:
        return 1
    else:
        return 0

In [126]:
if predict(['Hot', 'Mild', 'Rain', 'Weak']) == 1:
    print('Yes')
else:
    print('No')

Yes


<br>3) Usando los datos del archivo HoM.txt, entrenar el clasificador gaussiano bayesiano ingenuo de sklearn y usarlo para clasificar la persona con sexo desconocido que figura en ese archivo.<br>

In [149]:
df_HoM = pd.read_csv('HoM.txt')
att_unknown_sex = np.array([5, 130, 8])
df_HoM = df_HoM.iloc[:8]
df_HoM

,Person,height,weight,foot size
0,male,6,180.0,12.0
1,male,5.92,190.0,11.0
2,male,5.58,170.0,12.0
3,male,5.92,165.0,10.0
4,female,5,100.0,6.0
5,female,5.5,150.0,8.0
6,female,5.42,130.0,7.0
7,female,5.75,150.0,9.0


In [144]:
from sklearn.naive_bayes import GaussianNB

In [159]:
gnb = GaussianNB()
gnb.fit(df_HoM.drop('Person', axis=1), df_HoM.Person)
prediction = gnb.predict(att_unknown_sex.reshape(1,-1))
if prediction[0] == 'female':
    prediction[0] = 'mujer'
else:
    prediction[0] = 'hombre'
    
print('La persona con sexo desconocido es ' + prediction[0])

La persona con sexo desconocido es mujer
